## Import libraries

In [ ]:
import albumentations as A
import cv2
import os
import random

from matplotlib import pyplot as plt
%matplotlib inline

from typing import List, Tuple, Dict

## Generate multiple augmentations from single image

- [x] Read a label file
- [x] Extract class_id & bbox coordinates of each annotated object
- [x] apply both pixel & spatial augmentation
- [x] save the augmented images as well as bboxes & labels to the same directory with modified names
- [x] modularize the entire pipeline for multiple images & multiple augmentations
- [x] split the **images** & **labels** dataset into train, valid & test

## GLOBAL VARS

In [ ]:
IMAGES_DIR = "test_dir/harvest_day/train/images"
LABELS_DIR = "test_dir/harvest_day/train/labels"

SEED = 7
AUGMENTATIONS_COUNT = 3

category_id_to_name = {0: "blue", 1: "purple", 2: "red"}

In [ ]:
transform = A.Compose([
	A.HorizontalFlip(p=0.75),
  A.ISONoise(color_shift=(0.01, 0.07), intensity=(0.2, 0.6), p=0.8),
  A.CoarseDropout(min_holes=6, max_holes=12, max_height=50, max_width=50, min_height=10, min_width=10, p=0.5),
  A.Blur(blur_limit=5, p=0.66),
  A.RandomBrightnessContrast(brightness_limit=(-0.25,0.25), contrast_limit=(-0.25, 0.25), p=0.66),
  A.ShiftScaleRotate(shift_limit=(-0.06, 0.06), scale_limit=(-0.1, 0.1), rotate_limit=(-30, 30), p=0.5),
  A.GaussNoise(var_limit=(100.0, 200.0), p=0.2),
  A.ColorJitter(brightness=(0.6,1.50), contrast=(0.8,1.3), saturation=(0.75, 3.00), hue=(-0.05, 0.05), p=0.8),
  A.RandomSizedBBoxSafeCrop(640, 640, erosion_rate=0.2, interpolation=1, p=0.5),
  A.SafeRotate(limit=(-30,30), interpolation=1, border_mode=0, always_apply=False, p=0.5),
],
	bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['category_ids'],
    min_area=2000,
    min_visibility=0.33,
  ), 
	p=1,
)

## Functions

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img: cv2.Mat, bbox: List[float], class_name: str, color: Tuple = BOX_COLOR, thickness: int=2) -> cv2.Mat:
    """Visualizes a single bounding box on the image"""
    img_h, img_w, _ = img.shape
    x_center, y_center, w, h = bbox
    x_center = int(x_center * img_w)
    y_center = int(y_center * img_h)
    w = int(w * img_w)
    h = int(h * img_h)
    x_min, x_max, y_min, y_max = int(x_center-w/2), int(x_center + w/2), int(y_center-h/2), int(y_center + h/2)

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35,
        color=TEXT_COLOR,
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image: cv2.Mat, bboxes: List[List[float]], category_ids: List[str], category_id_to_name: Dict[int, str])->None:
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [ ]:
def parse_bboxes_n_labels(lines_read: List[str]) -> Tuple[List[List[float]], List[int]]:
	bboxes = []
	labels = []

	for line in lines_read:
		line = line.split(" ")
		label = line[0]
		bbox = line[1:]
		label = int(label)
		bbox = [float(x) for x in bbox]
		bboxes.append(bbox)
		labels.append(label)

	return bboxes, labels

In [ ]:
def get_img_n_labelTxt(file_name: str) -> Tuple[cv2.Mat, List[str]]:
	txt_file = os.path.join(LABELS_DIR, file_name + ".txt")
	img_file = os.path.join(IMAGES_DIR, file_name + ".jpg")

	img = cv2.imread(img_file)
	img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

	with open(txt_file, "r") as f:
		lines_read = f.readlines()
	
	lines_read = [line.strip() for line in lines_read]
	return img, lines_read

In [ ]:
def write_to_file(file_name: str, transformed) -> None:
	txt_file = os.path.join(LABELS_DIR, file_name )
	img_file = os.path.join(IMAGES_DIR, file_name )

	cv2.imwrite(img_file, cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR))

	with open(txt_file, "w") as f:
		for bbox, label in zip(transformed["bboxes"], transformed["category_ids"]):
			line = f"{label} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n"
			f.write(line)

## Main function

In [ ]:
def main()->None:
	files = os.listdir(IMAGES_DIR)

	for file in files:
		file_name = os.path.splitext(file)[0]

		print(file_name)

		img, lines_read = get_img_n_labelTxt(file_name)

		bboxes, category_ids = parse_bboxes_n_labels(lines_read)

		# visualize(img, bboxes, category_ids, category_id_to_name)
		SEED = 1

		for i in range(AUGMENTATIONS_COUNT):

			random.seed(SEED)
			SEED+=1

			transformed = transform(image=img, bboxes=bboxes, category_ids=category_ids)

			# visualize(transformed['image'], transformed['bboxes'], transformed['category_ids'], category_id_to_name)

			new_file_name = file_name + f"_aug_{i}"

			write_to_file(new_file_name, transformed)

		
	return

In [ ]:
main()